# Import packages
run the following cell to import necessary packages to run the analysis

In [18]:
import os
from pathlib import Path
from tqdm.notebook import tqdm
from bioio import BioImage
import bioio_czi
from pylibCZIrw import czi as pyczi
import numpy as np

# Reading data

In [2]:
data_path = Path('/media/npmartins/Data/nuno_martins/Image_data/NPM_Ex004_Re04/20241204_c2/NPM_Ex004_Re04_gel2_c2_Im-01_AcquisitionBlock1.czi/')
filelist = os.listdir(data_path)
print(len(filelist))

2


In [3]:
filelist = [x for x in filelist if x.find('.czi')>0]
filelist.sort()

In [4]:
file_index = 0
file_path = str(data_path)+'/'+filelist[file_index]
filename = filelist[file_index][:filelist[file_index].index('.czi')]
print(file_path)
print(filename)

/media/npmartins/Data/nuno_martins/Image_data/NPM_Ex004_Re04/20241204_c2/NPM_Ex004_Re04_gel2_c2_Im-01_AcquisitionBlock1.czi/NPM_Ex004_Re04_gel2_c2_Im-01_AcquisitionBlock1.czi
NPM_Ex004_Re04_gel2_c2_Im-01_AcquisitionBlock1


In [30]:
save_path = Path(str(data_path)+'/split_czi')
save_path.mkdir(parents=True, exist_ok=True)
print(save_path)

/media/npmartins/Data/nuno_martins/Image_data/NPM_Ex004_Re04/20241204_c2/NPM_Ex004_Re04_gel2_c2_Im-01_AcquisitionBlock1.czi/split_czi


___
with Zeiss library `pylibczirw`


In [45]:
with pyczi.open_czi(file_path) as czidoc:
    md_xml = czidoc.raw_metadata

# with open('./xml_metadata_airyscan.xml', 'w') as output:
#     output.write(md_xml)

In [82]:
with pyczi.open_czi(file_path) as czidoc:
    md_dic = czidoc.metadata
    # print(md_xml[:1000])
    dims_all = czidoc.scenes_bounding_rectangle_no_pyramid
    print(dims_all)
    bd = czidoc.scenes_bounding_rectangle
    print(bd)
    tbd = czidoc.total_bounding_box
    print(tbd)
    pixel_types = czidoc.pixel_types
    print(pixel_types)

{0: Rectangle(x=2143565, y=736187, w=3632, h=3632)}
{0: Rectangle(x=2143565, y=736187, w=3632, h=3632)}
{'T': (0, 1), 'Z': (0, 344), 'C': (0, 6), 'H': (0, 2), 'X': (2143565, 2147197), 'Y': (736187, 739819)}
{0: 'Gray8', 1: 'Gray8', 2: 'Gray8', 3: 'Gray8', 4: 'Gray8', 5: 'Gray8'}


In [81]:
channels_display = md_dic['ImageDocument']['Metadata']['DisplaySetting']['Channels']['Channel']
# len(channels_display), channels_display[1]
for i in range(len(channels_display)):
    print(channels_display[i])

{'@Id': 'Channel:0', '@Name': 'AF647-T1', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color': '#FFFF0014', 'OriginalColor': '#FFFF0014'}
{'@Id': 'Channel:1', '@Name': 'AF647#-T1', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color': '#FFFF0014', 'OriginalColor': '#FFFF0014'}
{'@Id': 'Channel:2', '@Name': 'AF568-T2', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color': '#FFFF6600', 'OriginalColor': '#FFFF6600'}
{'@Id': 'Channel:3', '@Name': 'AF568#-T2', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color': '#FFFF6600', 'OriginalColor': '#FFFF6600'}
{'@Id': 'Channel:4', '@Name': 'DAPI-T3', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color': '#FF00A1FF', 'OriginalColor': '#FF00A1FF'}
{'@Id': 'Channel:5', '@Name': 'DAPI#-T3', 'BitCountRange': '8', 'PixelType': 'Gray8', 'IlluminationType': 'Fluorescence', 'Color'

___
with `bioio` and `bioio-czi`


In [6]:
img = BioImage(file_path, reader=bioio_czi.Reader)
img.dims, img.dims.order

(<Dimensions [T: 1, C: 6, Z: 344, Y: 3632, X: 3632]>, 'TCZYX')

___ 
Read and write airyscan data

### save single channel for processing

In [22]:
def getDimensionPosition(dimension_order, dimension_name):
    return dimension_order.index(dimension_name)

In [39]:
with pyczi.open_czi(file_path) as czidoc_r:
    tbd = czidoc_r.total_bounding_box
    bd = czidoc_r.scenes_bounding_rectangle

    save_file_path = str(save_path)+'/'+filename+'_ch1'+'.czi'
    with pyczi.create_czi(save_file_path, exist_ok=True) as czidoc_w:
        for t in range(tbd['T'][-1]):
            for slice in tqdm(range(tbd['Z'][-1])):
                for channel in range(2):
                    for phase in range(tbd['H'][-1]):
                        slice_temp = czidoc_r.read(plane={'T': t, 'Z': slice, 'C': channel, 'H': phase})
                        czidoc_w.write(
                            data=slice_temp,
                            plane={
                                'T': t,
                                'Z': slice,
                                'C': channel,
                                'H': phase,
                            }
                        )




  0%|          | 0/344 [00:00<?, ?it/s]

from Zeiss' website

In [ ]:
# data = inference_service.run() # [2, y, x] array with 2 being the number of classes.

# with czi.open_czi(path, 'w'):

#     class_nuclei = data[0,:,:]
#     channel_0 = { 'C': 0 }

#     czi.write(data=class_nuclei, location=(0,0), plane=channel_0, compression_options = "zstd0:ExplicitLevel=0" )

#     class_background = data[1,:,:]
#     channel_1 = { 'C': 1 }

#     czi.write(data=class_background, location=(0,0), plane=channel_1, compression_options = "zstd1:ExplicitLevel=2" )

#     # If we stopped here, we'd have a valid CZI. But we want to name the channels after the classes.

#     channel_names = { 0: "C1", 1: "C2" }
#     custom_attributes = {"key1": "value1", "key2": "value2"}

#     czi.write_metadata(channel_names=channel_names, scale_x=0.1 * 1e-6, scale_y=0.1 * 1e-6, custom_attributes=custom_attributes)

#     # If we want to specify specific colors for specific channels, we write as follows:
#     # In this case we are specifying c1 as blue and c2 as green.
    
#     display_setting_dict: Dict[int, ChannelDisplaySettingsDataClass] = {}
#     tint_color_c1 = Rgb8Color(np.uint8(0x00), np.uint8(0xFF), np.uint8(0x00))
#     channel_setting_c1 = ChannelDisplaySettingsDataClass(True, TintingMode.Color, tint_color_c1)
#     display_setting_dict[0] = channel_setting_c1
#     tint_color_c2 = Rgb8Color(np.uint8(0x00), np.uint8(0x00), np.uint8(0xFF))
#     channel_setting_c2 = ChannelDisplaySettingsDataClass(True, TintingMode.Color, tint_color_c2)
#     display_setting_dict[1] = channel_setting_c2

#     czi.write_metadata(display_settings=display_setting_dict)

#     # Similarly, if we want to specify specific colors and black/white levels for specific channels, we write as follows:
#     # In this case we are specifying c1 as blue and c2 as green.
#     # Additionally we specify c1 with black point as 0.2 and white point as 0.8 and c2 with black point as 0.3 and white point as 0.75.

#     display_setting_dict: Dict[int, ChannelDisplaySettingsDataClass] = {}
#     tint_color_c1 = Rgb8Color(np.uint8(0x00), np.uint8(0xFF), np.uint8(0x00))
#     channel_setting_c1 = ChannelDisplaySettingsDataClass(True, TintingMode.Color, tint_color_c1, 0.2, 0.8)
#     display_setting_dict[0] = channel_setting_c1
#     tint_color_c2 = Rgb8Color(np.uint8(0x00), np.uint8(0x00), np.uint8(0xFF))
#     channel_setting_c2 = ChannelDisplaySettingsDataClass(True, TintingMode.Color, tint_color_c2, 0.3, 0.75)
#     display_setting_dict[1] = channel_setting_c2

#     czi.write_metadata(display_settings=display_setting_dict)

#     # Note: writing display setting for a channel overwrites any existing display setting as we do not fetch the current display setting.
#     # Note: There is no 1:1 relationship enforced. A user may decide to add display settings to each channel or only to some channels.
#     #       Similarly, it is not verified if the user sends more display settings than channels present.
#     #       Display setting that are not written will be set as 'empty' regardless of if the initially existed for that channel.